In [ ]:
import json
import csv
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import scipy.spatial.distance
#import nltk

In [ ]:
## Import data
with open('face-rating-data.json') as desc_file:
    descr = json.load(desc_file)
with open('guess-who-data.json') as data_file:    
    data = json.load(data_file)

In [ ]:
qa = np.genfromtxt("uid_qa.txt", delimiter = ",", names = True, dtype = [('int64'), ('int64'), ('int64'), ('U256'), ('U128'),])

In [ ]:
#qa[971]

In [ ]:
fe = np.genfromtxt("uid_pre_elim.txt", delimiter = ",", names = True, dtype = [('int64'), ('int64'), ('int64'), ('U256'), ('U256'),])

In [ ]:
fe.dtype.names = ('uniqueID', 'bn', 'qn', 'pre_que', 'curr_elim')

#fe.dtype.names
#fe[971]

In [ ]:
ftd = np.genfromtxt("face_id_descr.txt", delimiter = ";", skip_header = 1 , usecols = np.arange(0,2), dtype = [('U16'), ('U2056')])

In [ ]:
ftd.dtype.names = ('img_id', 'description')
len(ftd['img_id'])
len(set(ftd['img_id']))

In [ ]:
## first convert strings to list of rows equal to rows in fe
def mk_list(str_vector):
    des_byid = str_vector
    all_x = [i.split() for i in des_byid]
    #for i in des_byid:
    #    j = i.split()
        #if len(j) > 0:
    #    all_x.append(j)

    return all_x


In [ ]:
all_x = mk_list(fe['pre_que'])
#print(len(all_x)[1])
print(all_x[0])

In [ ]:
## Now for each row in the list above, find description and add it
all_i_txt = []

for rown in range(len(all_x)):
    #if rown not in bl:
        #print(all_x.index)
    int_i_txt = []
    for img in all_x[rown]:
        if img in ftd['img_id']:
            d = str(ftd['description'][list(ftd['img_id']).index(img)])
            int_i_txt.append(str(d))
    all_i_txt.append(int_i_txt)
        
            
            

In [ ]:
#print check
#print(type(all_i_txt[0]))
print(len(all_i_txt))
print(all_i_txt[971])

In [ ]:
## add question and answer to each data point
for it in range(len(all_i_txt)):
    all_i_txt[it].extend([qa['que'][it].strip(), qa['ans'][it].strip()])
    
    

In [ ]:
## check addition of above objects
#for i in range(len(all_i_txt)):
#    print(all_i_txt[i][-5:])

In [ ]:
#for i in set(bl):
#    print(all_i_txt[i][-2:])

In [ ]:
## Segregate indices where answers are yes or No
yes_ind = []
no_ind = []
for i in range(len(all_i_txt)):
    if all_i_txt[i][-1] == 'No':
        all_i_txt_no.append(all_i_txt[i])
        no_ind.append(i)
    else:
        all_i_txt_yes.append(all_i_txt[i])
        yes_ind.append(i)

In [ ]:
no_ind[0:5]

In [ ]:
print(len(all_i_txt))

In [ ]:
#print(all_i_txt[0])
##basic model
tf = TfidfVectorizer()

In [ ]:
## all similarities
all_sim = []
for l in range(len(all_i_txt)):
    H = tf.fit_transform(all_i_txt[l])
    sim = cosine_similarity(H)
    all_sim.append(sim)

In [ ]:
#similarities[-2] meaning taking only the array for question
all_sim_q = []
for i in range(len(all_sim)):
    all_sim_q.append(all_sim[i][-2][:-2])
print(len(all_sim_q))
#print(all_sim_q[971])
#print(len(all_sim))

In [ ]:

all_cheat = mk_list(fe['curr_elim'])


In [ ]:
len(all_cheat_yes)
all_cheat_yes[4]

In [ ]:
cheat_code = zip(all_sim_q, all_cheat, [i[-1] for i in all_i_txt])

In [ ]:
list(cheat_code)[3]

In [ ]:
## Sort all the above similarities with argsort which gives indices
## or we should not take top5 and make a threshold above which we will include it in the list of output?
sorted_ind = []

for si, ln, v in cheat_code:
    if v == "Yes":
        y_top = si.argsort()[::-1][:len(ln)]
        print(y_top)
    else:
        y_top = si.argsort()[:len(ln)]
        print(y_top)
    sorted_ind.append(y_top)
#sorted_ind[971]


In [ ]:
top_img = zip(all_x, sorted_ind)


In [ ]:
## convert to img_id from index to match with actual image ids selcted by participant
def mk_imgid(arr):
    y_pred = []
    for i,j in arr:
        yp = []
        for s in j:
            yp.append(i[s])
        y_pred.append(yp)
    return y_pred
    
    

In [ ]:
pred_img = mk_imgid(top_img)

In [ ]:
#print(all_cheat[0:3])
#v = mk_imgid(top_img)
#print(w)

In [ ]:
acc_in = zip(all_cheat, pred_img)

In [ ]:
acc_all = []
for a,p in acc_in:
    #print(a,p)
    if len(a) == 0:
        acc_i = 0
    else:
        acc_i = len(set(a).intersection(set(p)))/len(set(a))
    acc_all.append(acc_i)

In [ ]:
#len(pred_img)
print(acc_all)

In [ ]:
np.mean(np.array(acc_all))